# Applied Data Science Capstone
### Week 3 - Part 1 - Toronto Neighborhoods
#### Author: Rodrigo Eiras - rsveiras@gmail.com

### Installing required packages

In [1]:
!pip install beautifulsoup4
!pip install lxml
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

     |████████████████████████████████| 122kB 5.7MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 3.1MB/s eta 0:00:01     |███████████████████▋            | 3.4MB 3.1MB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         394 KB

The following NEW packages will be INSTALLED:

  geographiclib      co

### Importing libs

In [44]:
import requests
import urllib.request
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

### Downloading the wikipedia page

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
response

<Response [200]>

### Parsing the HTML table to a DataFrame

In [4]:
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find(name='table')

In [5]:
#soup.findAll('table')

In [6]:
table_str = str(table)
df = pd.read_html(table_str)[0]

In [7]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


##### Testing the GeoCode API using Rio de Janeiro as city

In [8]:
address = 'Copacabana, Rio de Janeiro, Brazil'

geolocator = Nominatim(user_agent="geo_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Copacabana, Rio de Janeiro are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Copacabana, Rio de Janeiro are -22.9719642, -43.1843432.


In [9]:
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### Drop NA values

In [10]:
# drop cells with a borough that is Not assigned
df_dropna = df[df.Borough != "Not assigned"].reset_index(drop=True)
df_dropna.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
df_dropna

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Group neighborhood in the same borough

In [12]:
df_grouped = df_dropna.groupby(["Postal Code", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Setting the Borough as Neighborhood for NA

In [13]:
for index, row in df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
df_grouped.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Handling Postal Codes

In [14]:
column_names = ["Postal Code", "Borough", "Neighborhood"]
pc_df = pd.DataFrame(columns=column_names)

pc_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in pc_list:
    pc_df = pc_df.append(df_grouped[df_grouped["Postal Code"]==postcode], ignore_index=True)
    
pc_df

,Postal Code,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


### Shapping the cleaned dataframe

In [23]:
df_grouped.shape

(103, 3)

In [25]:
df_grouped.to_csv('Toronto_Zipcodes.csv')

### Finished Part 1!